In [ ]:
# Import python packages
from sklearn import pipeline, preprocessing, ensemble, metrics
from snowflake.snowpark.context import get_active_session
from snowflake.ml.registry import Registry
import warnings
warnings.filterwarnings('ignore')

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# Set up the database, schema and model registry
database_name='DEV_AUTOMATION_DEMO'
schema_name='CHAMPION_CHALLENGER'
session.use_database(database_name)
session.use_schema(schema_name)    
# Initialize registry
registry = Registry(session=session, 
                           database_name=database_name, 
                           schema_name=schema_name)  


# Load the main dataset
This dataset was generated in the initial script and data was saved in the table named "Full_Data"

In [ ]:
#Load and convert to pandas dataframe
full_dataset = session.table('full_data').to_pandas()

# Time-Based Data Splits
```
Weeks 3-12:  Training Data (Challenger)
Weeks 13-15: Test Data 
Weeks 16-19:   Evaluation Data (Hold-out for comparison)

In [ ]:
def get_data(full_dataset, retrain_week_start = 3):
    
    # Use recent training data + some historical data for stability
    # BUT ensure challenger doesn't train on test data (avoid data leakage)
    historical_weeks = 10
    test_start_weeks = historical_weeks + retrain_week_start
    
    # Challenger trains from week 3 up to week 12 (before test data starts)
    challenger_train_mask = ((full_dataset['week_number'] >= retrain_week_start) & 
                       (full_dataset['week_number'] < retrain_week_start + historical_weeks))     
    challenger_train_data = full_dataset[challenger_train_mask].copy()
    print(f"   📊 Challenger training on {len(challenger_train_data):,} samples")
    print(f"   📅 Data from weeks {challenger_train_data['week_number'].min()}-{challenger_train_data['week_number'].max()}")
        
    #Three months of test data from Weeek 13 to 15
    challenger_test_mask = ((full_dataset['week_number'] >= test_start_weeks) & 
                       (full_dataset['week_number'] < test_start_weeks + 3))
    challenger_test_data = full_dataset[challenger_test_mask].copy()
    print(f"   📊 Challenger testing on {len(challenger_test_data):,} samples")
    print(f"   📅 Data from weeks {challenger_test_data['week_number'].min()}-{challenger_test_data['week_number'].max()}")
        
    return challenger_train_data, challenger_test_data
       
    

### Train challenger model using recent data (simulates weekly retraining)


In [ ]:
def train_challenger_model(challenger_train_data, challenger_test_data):
        
        print(f"🥊 Training Challenger Model...")
    
        # Feature columns (exclude date, week, and target)
        feature_cols = [col for col in challenger_train_data.columns 
                       if col not in ['application_date', 'week_number', 'approved']]

        X_challenger = challenger_train_data[feature_cols]
        y_challenger = challenger_train_data['approved']
        
        # Create challenger pipeline (could have different hyperparameters)
        challenger_pipeline = pipeline.Pipeline([
            ('scaler', preprocessing.StandardScaler()),
            ('classifier', ensemble.RandomForestClassifier(
                n_estimators=120,  # Slightly different config
                random_state=42,
                max_depth=12,
                min_samples_split=3,
                class_weight='balanced_subsample'  # Better for recent data
            ))
        ])
        
        # Train challenger
        challenger_pipeline.fit(X_challenger, y_challenger)
        
        # Evaluate challenger on the test set
        X_test = challenger_test_data[feature_cols]
        y_test = challenger_test_data['approved']
        
        challenger_pred_proba = challenger_pipeline.predict_proba(X_test)[:, 1]
        challenger_auc = round(metrics.roc_auc_score(y_test, challenger_pred_proba) * 100.0, 2)
        
        print(f"   ✅ Challenger trained successfully")
        print(f"   📈 Test AUC: {challenger_auc:.2f}")
        
        # Register challenger in model registry
        sample_input = X_challenger.head(100)
        model_name="CREDIT_APPROVAL"

        challenger_ref = registry.log_model(
            model = challenger_pipeline,
            model_name = model_name,
            sample_input_data = sample_input,
            target_platforms = ["WAREHOUSE", "SNOWPARK_CONTAINER_SERVICES"],
            comment = f"Challenger model retrained on recent data, AUC: {challenger_auc:.4f}",
            metrics = {
                "test_auc": challenger_auc,
                "train_weeks": f"{challenger_train_data['week_number'].min()}-{challenger_train_data['week_number'].max()}",
                "model_type": "challenger",
                "training_samples": len(X_challenger)
            }
        )

        try:
            challenger_ref.unset_alias("CHALLENGER")
        except:
            pass
        # Set as CHALLENGER alias
        challenger_ref.set_alias("CHALLENGER")
        model = registry.get_model(model_name)
        model.set_tag("CHALLENGER_VERSION", challenger_ref.version_name)
        
        print(f"   🏷️ Challenger registered: {challenger_ref.version_name}")
        print(f"   🏷️ Aliases: CHALLENGER")
    
        return 

In [ ]:
#Get the train and test data split for Challenger
train_data,test_data = get_data(full_dataset)
#Train the Challenger model and push into Snowflake model registry
train_challenger_model(train_data, test_data)